<a href="https://colab.research.google.com/github/AngeMervaneJdev/ML_AI_1/blob/main/Ensemble_Learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVR
from sklearn.model_selection import cross_val_score
from sklearn.decomposition import PCA
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVR
from sklearn.pipeline import make_pipeline
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score

In [5]:
data=pd.read_csv("train.csv")
test=pd.read_csv("test.csv")
X=np.array(data[['GrLivArea',"YearBuilt"]])
y=np.array(data.SalePrice)
y

array([208500, 181500, 223500, ..., 266500, 142125, 147500])

In [6]:
X

array([[1710, 2003],
       [1262, 1976],
       [1786, 2001],
       ...,
       [2340, 1941],
       [1078, 1950],
       [1256, 1965]])

#Probleme 1

In [7]:
X_train, X_test, y_train, y_test=train_test_split(X,y,random_state=42)

In [8]:
clf1 = LogisticRegression(multi_class='multinomial', random_state=1,solver='lbfgs', max_iter=50)
clf2 = RandomForestClassifier(n_estimators=50, random_state=1)
clf3 = GaussianNB()

In [9]:
models=[clf1,
clf2, clf3]

##1

In [10]:
#first 
predictions=[]
for ml in models:
  predictions.append(ml.fit(X_train,y_train).predict(X_test))
predictions= np.array(predictions)
MSE = mean_squared_error(y_test,np.mean(predictions,axis=0))
print('MSE : '+str(MSE))

eclf1 = VotingClassifier(estimators=[ ('lr', clf1), ('rf', clf2), ('gnb', clf3)], voting='hard')
eclf1 = eclf1.fit(X_train, y_train)
y_pred=eclf1.predict(X_test)

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


MSE : 2286391853.0849314


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


##2

In [11]:
#second
models[2]=SVR();
predictions=[]
for ml in models:
  predictions.append(ml.fit(X_train,y_train).predict(X_test))
predictions= np.array(predictions)
MSE = mean_squared_error(y_test,np.mean(predictions,axis=0))
print('MSE : '+str(MSE))

eclf1 = VotingClassifier(estimators=[ ('lr', clf1), ('rf', clf2), ('gnb', clf3)], voting='hard')
eclf1 = eclf1.fit(X_train, y_train)
y_pred=eclf1.predict(X_test)

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


MSE : 3121982673.3040857


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


##3

In [12]:
#third
models[1]=DecisionTreeClassifier();
models[2]=SVR();
predictions=[]
for ml in models:
  predictions.append(ml.fit(X_train,y_train).predict(X_test))
predictions= np.array(predictions)
MSE = mean_squared_error(y_test,np.mean(predictions,axis=0))
print('MSE : '+str(MSE))

eclf1 = VotingClassifier(estimators=[ ('lr', clf1), ('rf', clf2), ('gnb', clf3)], voting='hard')
eclf1 = eclf1.fit(X_train, y_train)
y_pred=eclf1.predict(X_test)


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


MSE : 3263714040.9951105


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


# Probleme 2

In [13]:
base_algorithms = [LogisticRegression(), DecisionTreeClassifier(),KNeighborsClassifier(n_neighbors=3)] #for classification

def bagging(X,y,models):
  stacking_train_dataset = []
  for model in models:
     X_train,X_test,y_train,y_test = train_test_split(X,y,random_state=42,shuffle=True)    
     stacking_train_dataset.append(model.fit(X_train,y_train).predict(X_test))
  return stacking_train_dataset;

bagging(X,y,base_algorithms)



/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


[array([110000, 290000, 110000, 190000, 190000,  80000, 190000, 140000,
         80000, 140000, 140000, 110000, 110000, 140000, 190000, 110000,
        190000, 110000, 110000, 140000, 190000, 140000, 140000, 110000,
        140000, 140000, 140000, 110000, 140000, 140000, 140000, 190000,
        485000, 110000, 190000, 110000, 190000, 190000, 290000, 110000,
        140000, 160000, 110000, 190000, 110000, 250000, 110000, 110000,
        250000, 140000, 110000, 140000, 140000, 290000, 110000, 190000,
        190000, 110000, 110000, 110000,  39300, 140000, 290000, 140000,
        190000, 160000, 110000, 160000, 110000, 140000, 140000, 140000,
        110000, 110000, 250000, 140000, 250000, 250000, 110000,  80000,
        110000, 110000, 110000, 110000, 140000, 190000, 190000, 140000,
        110000, 140000, 140000, 190000,  80000, 290000, 190000, 140000,
        140000, 140000, 140000, 250000, 190000, 190000, 160000, 110000,
        140000, 140000, 110000, 190000, 110000, 160000,  80000, 

#Probleme 3

In [14]:
x_train, x_test, y_train, y_test=train_test_split(X,y,random_state=42)
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.3,random_state=23)

In [16]:
def train_level_1( final_learner, train_meta_model, test_meta_model):
    # Train is carried out with final learner or meta model
    final_learner.fit(train_meta_model, y_val)

    # Getting train and test accuracies from meta_model
    print(f"Train accuracy: {final_learner.score(train_meta_model, y_val)}")
    print(f"Test accuracy: {final_learner.score(test_meta_model, y_test)}")

In [17]:
def train_level_0( clf):
        # Train with base x_train
        clf.fit(x_train, y_train)

        # Generate predictions for the holdout set (validation)
        # These predictions will build the input for the meta model
        val_predictions = clf.predict(x_val)

        # Generate predictions for original test set
        # These predictions will be used to test the meta model
        test_predictions = clf.predict(x_test)

        return val_predictions, test_predictions

In [18]:
weak_learners = [('dt', DecisionTreeClassifier()),('knn', KNeighborsClassifier()),('rf', RandomForestClassifier()),('gn', GaussianNB())]

# Final learner or meta model
final_learner = LogisticRegression()

train_meta_model = None
test_meta_model = None

# Start stacking
for clf_id, clf in weak_learners:

    # Predictions for each classifier based on k-fold
    val_predictions, test_predictions = train_level_0(clf)

    # Stack predictions which will form 
    # the input data for the data model
    if isinstance(train_meta_model, np.ndarray):
        train_meta_model = np.vstack((train_meta_model, val_predictions))
    else:
        train_meta_model = val_predictions

    # Stack predictions from test set
    # which will form test data for meta model
    if isinstance(test_meta_model, np.ndarray):
        test_meta_model = np.vstack((test_meta_model, test_predictions))
    else:
        test_meta_model = test_predictions

# Transpose train_meta_model
train_meta_model = train_meta_model.T

# Transpose test_meta_model
test_meta_model = test_meta_model.T

# Training level 1
train_level_1(final_learner, train_meta_model, test_meta_model)

Train accuracy: 0.1762917933130699
Test accuracy: 0.005479452054794521


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,
